In [1]:
import tools
corpus = tools.load_corpus()
import pickle as pkl
from importlib import reload as rl
import non_iid_dataloader as niiddl, lhotse
from tqdm import tqdm
from torch.nn import TransformerDecoder, TransformerDecoderLayer, Transformer
import torch
from omegaconf.omegaconf import OmegaConf
import matplotlib.pyplot as plt
import numpy as np
import einops
from torch import einsum


[NeMo W 2022-11-15 11:43:26 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-11-15 11:43:28 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-11-15 11:43:28 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-11-15 11:43:28 nemo_logging:349] /exp/exp1/acp21rjf/lhotse/lhotse/lazy.py:388: UserWarning: A lambda was passed to LazyMapper: it may prevent you from forking this process. If you experience issues with num_workers > 0 in torch.utils.data.DataLoader, try passing a regular function instead.
      warnings.warn(
    


In [2]:
import model_utils

In [3]:
!ls

checkpoint_qknorm_60s_inter_50do_128  model_utils.py
checkpoint_qknorm_ctx_temp	      ngrams
checkpoint_qknorm_hierarchy_do50      non_iid_dataloader.py
create_tokenizer		      plain_text
eval_ctc_h_init.py		      __pycache__
eval_ctc_reuse.py		      tokenizers
eval_perplexity_pretrained.py	      tools.py
eval_perplexity.py		      train_H.py
generate_multi_hypothesis.py	      train_LM.py
hypothesis.pkl			      train_lm.sh
lm				      train_long_context.sh
lm_exps				      train_roberta.sh
lm_utils.py			      Untitled-1.ipynb
log_probs_0.pkl			      wandb
log.txt


In [5]:
with open('log_probs_0.pkl', 'rb') as f:
    log_probs = pkl.load(f)

In [6]:
tk = tools.load_tokenizer('./tokenizers/tokenizer_spe_bpe_v128/tokenizer.model')

In [7]:
from importlib import reload as rl
rl(model_utils)
rl(niiddl)
import non_iid_dataloader as niiddl

In [8]:
from pyctcdecode import build_ctcdecoder

def kenlm_decoder(arpa_, vocab, alpha=0.5, beta=0.8):  
    arpa = arpa_ if arpa_ != '' else None
    alpha = alpha if arpa_ != '' else None
    beta = beta if arpa_ != '' else None
    decoder = build_ctcdecoder(vocab, kenlm_model_path=arpa, alpha=alpha, beta=beta)
    print(f'Loaded KenLM model from {arpa} with alpha={alpha} and beta={beta}')
    return decoder

In [9]:
lm = kenlm_decoder('./ngrams/cantab_interp_tedlium.arpa', tk.vocab)

Loading the LM will be faster if you build a binary file.
Reading /exp/exp1/acp21rjf/deliberation/speachy/tedlium/ngrams/cantab_interp_tedlium.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigrams and labels don't seem to agree.


Loaded KenLM model from ./ngrams/cantab_interp_tedlium.arpa with alpha=0.5 and beta=0.8


In [10]:
log_probs.keys()

dict_keys(['log_probs', 'decoder', 'encoded_len'])

In [11]:
log_probs['encoded_len'].tolist()

[275]

In [140]:
dbeams = model_utils.decode_beams_lm(logits_list=log_probs['log_probs'], decoder=lm, encoded_lengths=log_probs['encoded_len'].tolist(), beam_width=20000)

In [12]:
dl = niiddl.get_eval_dataloader(corpus['test'], max_duration=3.0, return_speaker=True, return_meta_data=True)

In [171]:
for z in dl:
    a = z
    break

In [75]:
a['metadata'][0][0]

KeyError: 'metadata'

In [63]:
from nemo.collections.asr.metrics.wer import word_error_rate

In [4]:
with open('hypothesis.pkl', 'rb') as f:
    hypothesis = pkl.load(f)

In [69]:
from tqdm import tqdm

In [70]:
def sort_hypothesis_by_recording(hyps):
    recordings = {}
    for hyp in hyps:
        rec = hyp['meta_data']['recording_id']
        if rec not in recordings:
            recordings[rec] = []
        recordings[rec].append(hyp)
    return recordings

def order_recordings_by_start_time(hypothesis):
    for key in hypothesis.keys():
        hypothesis[key] = sorted(hypothesis[key], key=lambda x: x['meta_data']['timings']['segment_start'])
    return hypothesis

def get_oracle_wer(hypothesis):
    hyps, refs = [], []
    for key in hypothesis.keys():
        recording = hypothesis[key]
        for utt in tqdm(recording):
            best_idx, best_score = None, None
            target = utt['targets'][0]
            for idx in utt['beams'][0].keys():
                cur = utt['beams'][0][idx]
                hyptext = cur['text']
                hypwer = word_error_rate([hyptext], [target])
                if best_idx is None or hypwer < best_score:
                    best_idx = idx
                    best_score = hypwer
            hyps.append(utt['beams'][0][best_idx]['text'])
            refs.append(target)
    return word_error_rate(hyps, refs)

In [27]:
a = sort_hypothesis_by_recording(hypothesis)

In [28]:
a = order_recordings_by_start_time(a)

In [92]:
a.keys()

dict_keys(['AimeeMullins_2009P', 'EricMead_2009P', 'MichaelSpecter_2010', 'GaryFlake_2010', 'DanBarber_2010', 'DanielKahneman_2010', 'BillGates_2010', 'RobertGupta_2010U', 'JaneMcGonigal_2010', 'JamesCameron_2010', 'TomWujec_2010U'])

In [97]:
torch.tensor(a['AimeeMullins_2009P'][1]['beams'][0][0]['score']).exp()

tensor(0.5114, dtype=torch.float64)

In [86]:
v,b =a['AimeeMullins_2009P'][1]['meta_data']['timings'].values()

In [74]:
o = get_oracle_wer(a)

100%|██████████| 35/35 [00:00<00:00, 422.69it/s]


In [32]:
word_error_rate(['hello'],['bye'],)

1.0

In [56]:
a['AimeeMullins_2009P'][0]['beams'][0][0]

{'text': "i'd like to share with you a discovery that i made a few months ago while writing an article for the italian wired i always keep my the sars hand never'm writing anything but",
 'score': -5.665437203736285}

In [4]:
meetings = niiddl.prepare_partition(corpus['train'])

In [5]:
import lhotse
import non_iid_dataloader as niiddl
rl(niiddl),rl(lhotse)
from lhotse.dataset.collation import collate_audio
from lhotse.dataset.cut_transforms import plain_concat, individual_speaker_concat
niiddl.plain_concat = plain_concat
niiddl.individual_speaker_concat = individual_speaker_concat

samples = niiddl.prepare_samples(meetings, max_allowed_utterance_gap=8.0, max_duration=0)


In [6]:
rl(lm_utils)
model = lm_utils.load_model(lm_utils.load_config('./lm/decoder_test.yaml'), tools.load_tokenizer(), max_len=1862)

In [23]:
model.to('cpu')

transformer_lm(
  (layers): transformer(
    (positional_bias): DynamicPositionBias(
      (mlp): ModuleList(
        (0): Sequential(
          (0): Linear(in_features=1, out_features=64, bias=True)
          (1): Identity()
          (2): ReLU()
        )
        (1): Sequential(
          (0): Linear(in_features=64, out_features=64, bias=True)
          (1): Identity()
          (2): ReLU()
        )
        (2): Linear(in_features=64, out_features=8, bias=True)
      )
    )
    (layers): ModuleList(
      (0): ModuleList(
        (0): PreNorm(
          (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (fn): MyopicAttention(
            (qkv_proj): Linear(in_features=256, out_features=768, bias=False)
            (out_proj): Linear(in_features=256, out_features=256, bias=False)
          )
        )
        (1): PreNorm(
          (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (fn): Sequential(
            (0): GLU(
              (act

In [7]:
!ls checkpoints_LM3

ls: cannot access 'checkpoints_LM3': No such file or directory


In [8]:
rl(lm_utils), rl(model_utils)

(<module 'lm_utils' from '/exp/exp1/acp21rjf/deliberation/speachy/tedlium/lm_utils.py'>,
 <module 'model_utils' from '/exp/exp1/acp21rjf/deliberation/speachy/tedlium/model_utils.py'>)

In [9]:
epoch, val_loss  = model_utils.load_checkpoint(args=argsclass(**{'checkpoint': './checkpoint_myopic_test/checkpoint_11_id_18.pt'}), model=model, force_cpu=True)

./checkpoint_myopic_test/checkpoint_11_id_18.pt
odict_keys(['layers.positional_bias.mlp.0.0.weight', 'layers.positional_bias.mlp.0.0.bias', 'layers.positional_bias.mlp.1.0.weight', 'layers.positional_bias.mlp.1.0.bias', 'layers.positional_bias.mlp.2.weight', 'layers.positional_bias.mlp.2.bias', 'layers.layers.0.0.norm.weight', 'layers.layers.0.0.norm.bias', 'layers.layers.0.0.fn.qkv_proj.weight', 'layers.layers.0.0.fn.out_proj.weight', 'layers.layers.0.1.norm.weight', 'layers.layers.0.1.norm.bias', 'layers.layers.0.1.fn.0.proj.weight', 'layers.layers.0.1.fn.0.proj.bias', 'layers.layers.0.1.fn.2.weight', 'layers.layers.0.1.fn.2.bias', 'layers.layers.1.0.norm.weight', 'layers.layers.1.0.norm.bias', 'layers.layers.1.0.fn.qkv_proj.weight', 'layers.layers.1.0.fn.out_proj.weight', 'layers.layers.1.1.norm.weight', 'layers.layers.1.1.norm.bias', 'layers.layers.1.1.fn.0.proj.weight', 'layers.layers.1.1.fn.0.proj.bias', 'layers.layers.1.1.fn.2.weight', 'layers.layers.1.1.fn.2.bias', 'layers.laye

In [10]:
torch.cuda.is_available()

True

In [16]:
lm_utils.greedy_generate(
    model=model,
    tokenizer=tokenizer,
    input_txt='So I was speaking to Bill Ga',
    max_len=100,
    force_cpu=True,
    temperature=0.5
)

'⁇  so i was speaking to bill gave it each one of the most people and applied to it and then they were still a city looking up the streaming camp this was the high deposit thought that the particular reg'

In [27]:
model.greedy_generate('A prominent limitation of ', tokenizer=tokenizer, temperature=1.0)

NotImplementedError: 

In [24]:
import non_iid_dataloader as niiddl
import lm_utils
rl(niiddl)
rl(lm_utils)

<module 'lm_utils' from '/exp/exp1/acp21rjf/deliberation/speachy/tedlium/lm_utils.py'>

In [25]:
durations = [
    0.0,
    15.0,
    30.0,
    45.0,
    60.0,
    75.0,
    90.0,
    100.0,
    120.0,
    140.0,
    180.0,
    200.0,
    250.0,
    300.0,
    400.0,
    500.0,
]
for duration in durations:
    dl = niiddl.get_data_loader(
        corpus['test'], 
        tokenizer=tokenizer, 
        batch_size=100, 
        shuffle=False,
        max_duration=duration,
        text_only=True,
    )
    ppl, avg_len = lm_utils.eval_corpus_perplexity(model, dl, device='cpu')
    print(f"Duration: {duration}, PPL: {ppl}, Avg Len: {avg_len}")

loss: 0.70: 100%|██████████| 12/12 [00:19<00:00,  1.59s/it]


Duration: 0.0, PPL: 2.19421648979187, Avg Len: 67.19134521484375


loss: 0.72:  22%|██▏       | 2/9 [00:03<00:13,  1.90s/it]


KeyboardInterrupt: 

In [8]:
dl = niiddl.get_data_loader(
    corpus['train'], 
    tokenizer=tokenizer, 
    batch_size=20, 
    shuffle=True,
    max_duration=100,
    text_only=True,
)

In [9]:
for batch in dl:
    z = batch
    break

In [ ]:
x = lm_utils.eval_corpus_perplexity(model, dl, device='cpu')
x

In [436]:
#ppl, avg_len = lm_utils.eval_corpus_perplexity(model, dl, device='cpu')
#print(ppl, avg_len)

perplexity: 417.11: 100%|██████████| 26/26 [01:22<00:00,  3.17s/it] 

479.1307067871094 1441.75


In [339]:
lm_utils.eval_corpus_perplexity(model, dl, device='cpu')

perplexity: 459.58: 100%|██████████| 124/124 [00:18<00:00,  6.71it/s] 


167.10165405273438

In [158]:
lm_utils.greedy_generate(model, tokenizer, 'how are', 10, force_cpu=True, temperature=0)

[0, 17, 54, 97, 3, 107, 3, 107, 3, 107]


'⁇  how are as as as'

In [ ]:
model

In [159]:
rl(lm_utils)

<module 'lm_utils' from '/exp/exp1/acp21rjf/deliberation/speachy/tedlium/lm_utils.py'>

In [68]:
tokenizer.text_to_ids('hello there')

[78, 30, 104, 6, 8]

In [75]:
tokenizer.add_special_tokens(['<sos>', '<eos>'])

In [ ]:
tokenizer.additional_special_tokens_ids()

In [74]:
tokenizer.ids_to_text(out[0)

'en'

In [5]:
tokens = torch.randn(2,3)
token_lens = torch.tensor([2, 3])

NameError: name 'torch' is not defined

In [177]:
def add_eos(tkn, tkn_len, eos_id = 30):
    tkn = tkn.clone() 
    tkn[torch.arange(tkn.shape[0], device=tkn_len.device, dtype=torch.int32), (tkn_len-1.0).to(torch.int32)] = eos_id
    return tkn

token_lens.to(torch.int64)

tensor([2, 3])

In [181]:
[0] + tokenizer.text_to_ids('hello')

[0, 78, 30, 104]

In [178]:
add_eos(tokens, token_lens)

IndexError: tensors used as indices must be long, byte or bool tensors

In [145]:
bt = torch.tensor(
    [[0,0,0],
    [0,0,0]]
, dtype=torch.bool)

In [146]:
tokens[~bt]

tensor([ 0.3837,  0.5211, -1.2230,  0.5048, -0.1723, -2.0018])

In [39]:
from lm.s4 import S4

In [43]:
model  =S4(d_model=256)

In [54]:
emb = torch.nn.Embedding(100, 256)

In [58]:
emb(torch.randint(0, 100, (2,3))).shape

torch.Size([2, 3, 256])

In [61]:
from einops import rearrange



In [143]:
rl(lm_utils)
from lm_utils import S4adapter

In [301]:
get_num_params(
    S4adapter(S4(d_model=712, measure='legs', mode='nplr', transposed=False, d_state=64), vocab_size=128)
) / 1e6 

1.381408

In [139]:
model = S4adapter(S4(d_model=2048, measure='legs', mode='nplr', transposed=False, d_state=512), vocab_size=128)

In [140]:
model

S4adapter(
  (model): S4(
    (kernel): SSKernel(
      (kernel): SSKernelNPLR()
    )
    (activation): GELU()
    (dropout): Identity()
    (output_linear): Sequential(
      (0): Linear(in_features=2048, out_features=4096, bias=True)
      (1): GLU(dim=-1)
    )
  )
  (embedding): Embedding(128, 2048)
  (predict): Linear(in_features=2048, out_features=128, bias=True)
  (softmax): Softmax(dim=-1)
)

In [134]:

model(torch.randint(0, 100, (2,3))).shape

torch.Size([2, 3, 2048]) emb
torch.Size([2, 3, 2048]) s4
torch.Size([2, 3, 128]) logits


torch.Size([2, 3, 128])

In [12]:
import torch, torch.nn as nn

In [45]:
class GumbelSigmoid(nn.Module):
    """
    adapted : https://github.com/yandexdataschool/gumbel_lstm/blob/master/gumbel_sigmoid.py

    A gumbel-sigmoid nonlinearity with gumbel(0,1) noize
    In short, it's a function that mimics #[a>0] indicator where a is the logit
    
    Explaination and motivation: https://arxiv.org/abs/1611.01144
    
    Math:
    Sigmoid is a softmax of two logits: a and 0
    e^a / (e^a + e^0) = 1 / (1 + e^(0 - a)) = sigm(a)
    
    Gumbel-sigmoid is a gumbel-softmax for same logits:
    gumbel_sigm(a) = e^([a+gumbel1]/t) / [ e^([a+gumbel1]/t) + e^(gumbel2/t)]
    where t is temperature, gumbel1 and gumbel2 are two samples from gumbel noize: -log(-log(uniform(0,1)))
    gumbel_sigm(a) = 1 / ( 1 +  e^(gumbel2/t - [a+gumbel1]/t) = 1 / ( 1+ e^(-[a + gumbel1 - gumbel2]/t)
    gumbel_sigm(a) = sigm([a+gumbel1-gumbel2]/t)
    
    For computation reasons:
    gumbel1-gumbel2 = -log(-log(uniform1(0,1)) +log(-log(uniform2(0,1)) = -log( log(uniform2(0,1)) / log(uniform1(0,1)) )
    gumbel_sigm(a) = sigm([a-log(log(uniform2(0,1))/log(uniform1(0,1))]/t)
    
    
    :param t: temperature of sampling. Lower means more spike-like sampling. Can be symbolic.
    :param eps: a small number used for numerical stability
    :returns: a callable that can (and should) be used as a nonlinearity
    
    """
    def __init__(
            self, 
            init_temperature=1.0,
            final_temperature=0.1,
            total_steps=10000, 
            eps=1e-20,
            
        ):
        super().__init__()
        self.temperature=init_temperature
        self.final_temperature=final_temperature
        self.total_steps=total_steps
        self.decay = (final_temperature/init_temperature)**(1/total_steps)
        self.eps=eps

    def step(self):
        if self.temperature > self.final_temperature:
            self.temperature *= self.decay
         
    def __call__(self,logits):
        """computes a gumbel sigmoid sample"""
        temperature = self.temperature if self.training else self.final_temperature
        
        #sample from Gumbel(0, 1)
        uniform1 = torch.rand_like(logits)
        uniform2 = torch.rand_like(logits)
        
        noise = -torch.log(torch.log(uniform2 + self.eps)/torch.log(uniform1 + self.eps) +self.eps)
        #draw a sample from the Gumbel-Sigmoid distribution
        gumbel = ((logits + noise) / temperature).sigmoid()
        if self.training:
            self.step()
        return gumbel

In [231]:
gb = GumbelSigmoid(final_temperature=0.6)
gb.eval()

GumbelSigmoid()

In [251]:
rnd = torch.distributions.exponential.Exponential(0.1).sample(torch.Size([100]))
rnd = rnd / rnd.sum()

(gb(rnd).norm())

tensor(6.5119)

In [ ]:
import torch
from einops import rearrange, repeat
from functools import partial

In [9]:
q = torch.randn(10, 73, 256)
k = torch.randn(10, 73, 256)
temp = 256**0.5
pos_emb = torch.randn(73, 73)

In [14]:
fused = torch.baddbmm(pos_emb, q, k.transpose(-1, -2), beta=1.0, alpha=temp)
dots = torch.einsum('bnd,bmd->bnm', q, k) * temp + pos_emb
torch.allclose(fused, dots)

True

In [ ]:
## now with multi-heads, tricky bit is we need to make it batched i.e 3d, this means moving the head dim to the batch dim
q = torch.randn(10, 8, 73, 32)
k = torch.randn(10, 8, 73, 32)
temp = 32**0.5
pos_emb = torch.randn(8, 73, 73)

#qb = rearrange(q, 'b h n d -> (b h) n d')
#kb = rearrange(k, 'b h n d -> (b h) n d')
qb,kb = map(partial(rearrange, 'b h n d -> (b h) n d'), (q,k))
pos_emb_b = repeat(pos_emb, 'h n m -> (b h) n m', b=10)

fused = torch.baddbmm(pos_emb_b, qb, kb.transpose(-1, -2), beta=1.0, alpha=temp)
dots = torch.einsum('b h n d, b h m d -> b h n m', q, k) * temp + pos_emb

print(fused.shape, dots.shape)
fused = rearrange(fused, '(b h) n m -> b h n m', b=10, h=8)
torch.allclose(fused, dots)

torch.Size([80, 73, 73]) torch.Size([10, 8, 73, 73])


True

In [31]:
pos_emb.shape

torch.Size([80, 73, 73])

In [26]:
qb[1]

tensor([[ 0.6948, -1.2061,  0.6850,  ...,  1.0949,  0.7656,  0.3566],
        [ 0.8898,  1.1649, -2.3278,  ...,  0.5976,  0.9025, -1.4097],
        [-0.5865, -0.4260,  0.6259,  ..., -0.1008, -1.5885,  0.6942],
        ...,
        [ 0.2545,  0.1986,  0.7685,  ...,  1.5268,  0.5201,  1.5483],
        [ 0.4820,  0.7004,  0.7175,  ...,  0.1674,  0.6084,  0.1248],
        [-0.0952, -2.6570,  0.1465,  ..., -0.2255,  0.7898,  0.1224]])

In [25]:
q[0][1]

tensor([[ 0.6948, -1.2061,  0.6850,  ...,  1.0949,  0.7656,  0.3566],
        [ 0.8898,  1.1649, -2.3278,  ...,  0.5976,  0.9025, -1.4097],
        [-0.5865, -0.4260,  0.6259,  ..., -0.1008, -1.5885,  0.6942],
        ...,
        [ 0.2545,  0.1986,  0.7685,  ...,  1.5268,  0.5201,  1.5483],
        [ 0.4820,  0.7004,  0.7175,  ...,  0.1674,  0.6084,  0.1248],
        [-0.0952, -2.6570,  0.1465,  ..., -0.2255,  0.7898,  0.1224]])